In [ ]:
def getListedCode(typ="twse",codeLength=4):
    import sqlite3
    sqlite_file = "list.db" #上市上櫃下市 股票代號列表 資料庫檔案
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    out = conn.execute("SELECT code, type FROM StockList WHERE type LIKE ? AND length(code) == ? ",("twse",codeLength)) 
    # 取 代號4碼 類別上市 股票代號
    codeList = []
    for i, row in enumerate(out):
        codeList.append(row[0])
    conn.close()
    print("取得股票代號總數:", len(codeList)) #codeList 為取出股票代號列表
    return codeList
print("getListedCode loaded")

codeList = getListedCode()

def getStrategyData(code,startdate,enddate,dbname="buysell.db",table="twse"):
    import sqlite3
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    out = conn.execute("SELECT * FROM twse WHERE code LIKE ? AND date between ? and ?",(code,startdate,enddate))
    dataList = []
    for i in out:
        dataList.append(i)
    conn.close()
    return dataList

dataList = getStrategyData("1101","2016-05-25","2017-05-25")
lastData = getStrategyData("1101","2017-05-24","2017-05-25","PriceVolData.db")
print("lastData:",lastData[0][6])
sellcount = 0
buycount = 0
selldata = []
buydata = []
for i in dataList:
    #print("Code:",i[1],"Open:",i[6],"Action:",i[8])
    if "sell" == i[8]:
        sellcount = sellcount + 1
        selldata.append([i[1],i[6],i[8]])
    if "buy" == i[8]:
        buycount = buycount + 1
        buydata.append([i[1],i[6],i[8]])

print("Buy:",buydata,"Sell:",selldata)
print("Buy:",buycount,"張 Sell:",sellcount,"張")

profit = 0
for buy,sell in zip(buydata,selldata):
    profit = profit + (sell[1] - buy[1])
    buydata.pop()
    selldata.pop()
    print("Profit:",profit,"Buy:",buy[1],"Sell:",sell[1])
if buycount > sellcount:
    for i in buydata:
        profit = profit + (lastData[0][6]-i[1])
        print("Profit:",profit,"Clear:",lastData[0][6],"Buy:",i[1])
print(profit)

In [17]:
def getListedCode(typ="twse",codeLength=4):
    import sqlite3
    sqlite_file = "list.db" #上市上櫃下市 股票代號列表 資料庫檔案
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    out = conn.execute("SELECT code, type FROM StockList WHERE type LIKE ? AND length(code) == ? ",("twse",codeLength)) 
    # 取 代號4碼 類別上市 股票代號
    codeList = []
    for i, row in enumerate(out):
        codeList.append(row[0])
    conn.close()
    print("取得股票代號總數:", len(codeList)) #codeList 為取出股票代號列表
    return codeList

def getStrategyData(code,startdate,enddate,dbname="buysell.db",table="twse"):
    import sqlite3
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    out = conn.execute("SELECT * FROM twse WHERE code LIKE ? AND date between ? and ?",(code,startdate,enddate))
    dataList = []
    for i in out:
        dataList.append(i)
    conn.close()
    return dataList

def getLastDate(code,dbname="PriceVolData.db",table="twse"):
    import sqlite3
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    out = conn.execute("SELECT * FROM twse WHERE code LIKE ? ORDER BY date DESC LIMIT 1",(code,))
    dataList = []
    for i in out:
        dataList.append(i)
    conn.close()
    return dataList

def getStats(dataList,lastData):
    sellcount = 0
    buycount = 0
    selldata = []
    buydata = []
    for i in dataList:
        #print("Code:",i[1],"Open:",i[6],"Action:",i[8])
        if "sell" == i[8]:
            sellcount = sellcount + 1
            selldata.append([i[1],i[6],i[8]])
        if "buy" == i[8]:
            buycount = buycount + 1
            buydata.append([i[1],i[6],i[8]])

    #print("Buy:",buydata,"Sell:",selldata)
    #print("Buy:",buycount,"張 Sell:",sellcount,"張")

    profit = 0
    win = 0
    lose = 0
    costList = []
    for buy,sell in zip(buydata,selldata):
        difference = sell[1] - buy[1]
        costList.append(["Code: {}".format(dataList[0][1]),"Buy: {}".format(buy[1]),"Profit: {}".format(difference)])
        profit = profit + difference
        if 0 < (sell[1] - buy[1]):
            win = win + 1
        else:
            lose = lose + 1
        buydata.pop()
        selldata.pop()
        #print("Profit:",profit,"Buy:",buy[1],"Sell:",sell[1])
    if buycount > sellcount:
        #print(lastData)
        for i in buydata:
            difference = lastData[0][6]-i[1]
            costList.append(["Code: {}".format(dataList[0][1]),"Buy: {}".format(i[1]),"Profit: {}".format(difference)])
            profit = profit + difference
            if 0 < (lastData[0][6]-i[1]):
                win = win + 1
            else:
                lose = lose + 1
            #print("Profit:",profit,"Clear:",lastData[0][6],"Buy:",i[1])
    else:
        for i in selldata:
            difference = lastData[0][6]-i[1]
            costList.append(["Code: {}".format(dataList[0][1]),"Sell: {}".format(i[1]),"Profit: {}".format(difference)])
            profit = profit + difference
            if 0 < (lastData[0][6]-i[1]):
                win = win + 1
            else:
                lose = lose + 1
            #print("Profit:",profit,"Clear:",lastData[0][6],"Buy:",i[1])
    return profit,win,lose,costList

profitList = []
profitSum = 0
winSum = 0
loseSum = 0
codeList = getListedCode()
costListData = []
for code in codeList:
    dataList = getStrategyData(code,"2016-05-25","2017-05-25")
    lastData = getLastDate(code,dbname="PriceVolData.db",table="twse")
    profit,win,lose,costList = getStats(dataList,lastData)
    costListData.append(costList)
    profitList.append(["Code:{}".format(code),"Profit:{}".format(profit)])
    profitSum = profitSum + profit
    winSum = winSum + win
    loseSum = loseSum + lose
    
import pickle
pickle.dump(costListData,open('costListData.pkl', 'wb'))
print("Profit Sum:",profitSum,"Win Sum:",winSum,"Lose Sum:",loseSum)

取得股票代號總數: 913
Profit Sum: 15752.909999999996 Win Sum: 4431 Lose Sum: 3474


In [3]:
for i in costListData[1][0:50]:
    print(i)

NameError: name 'costListData' is not defined

In [7]:
def createDatabase(dbName,tableName):
    import os
    if os.path.isfile(dbName):
        return "Database name already exists: ", dbName
    else:    
        import sqlite3
        sqlite_file = dbName
        conn = sqlite3.connect(sqlite_file)
        c = conn.cursor()
        c.execute('CREATE TABLE {tn} (id INTEGER PRIMARY KEY, code TEXT NOT NULL, type TEXT NOT NULL, cost REAL, profit REAL)'.format(tn=tableName)) 
        conn.commit()
        conn.close()
        return "Database",dbName," Successfully Created!"
    
createDatabase("ProfitCosty1617.db","twseprofit")

('Database', 'ProfitCosty1617.db', ' Successfully Created!')

In [8]:
def insertData(dbName, costListData):
    import sqlite3
    conn = sqlite3.connect(dbName)
    c = conn.cursor()
    count = 0
    errCost = 0
    for codes in costListData:
        for quote in codes:
            code = quote[0][6:]
            profit = quote[2][8:]
            if 'B' == quote[1][0]:
                typ = "Buy"
                cost = quote[1][5:]
            elif 'S' == quote[1][0]:
                typ = "Sell"
                cost = quote[1][6:]
            else:
                print("Code:", quote, "Type Error", quote)
            if not float(cost):
                errCost = errCost+1
            else:
                conn.execute("INSERT INTO twseprofit (code, type, cost, profit) VALUES (?,?,?,?)",(code, typ, cost, profit))
                count = count + 1
    conn.commit()
    conn.close()
    return count, errCost

import pickle
costListData = pickle.load(open('costListData.pkl', 'rb'))
count, errCost = insertData("ProfitCosty1617.db", costListData)
print("count",count,"errCost",errCost)

count 7800 errCost 105


In [47]:
for codes in costListData:
    for quote in codes:
        code = quote[0]#[6:]
        profit = quote[2][8:]
        if 'B' == quote[1][0]:
            typ = "Buy"
            cost = quote[1][5:]
        elif 'S' == quote[1][0]:
            typ = "Sell"
            cost = quote[1][6:]
        else:
            print("Code:", quote[0], "Type Error", quote)
        print(code,"|", typ,"|", cost,"|", profit)

Code: C Type Error Code: 1102
C | Sell | 30.8 | 
Code: B Type Error Buy: 28.7
B | Sell | 30.8 | 
Code: P Type Error Profit: -2.9499999999999993
P | Sell | 30.8 | 
Code: C Type Error Code: 1102
C | Sell | 30.8 | 
Code: B Type Error Buy: 28.0
B | Sell | 30.8 | 
Code: P Type Error Profit: 1.1000000000000014
P | Sell | 30.8 | 
Code: C Type Error Code: 1102
C | Sell | 30.8 | 
Code: B Type Error Buy: 26.65
B | Sell | 30.8 | 
Code: P Type Error Profit: 2.3500000000000014
P | Sell | 30.8 | 
Code: C Type Error Code: 1102
C | Sell | 30.8 | 
Code: S Type Error Sell: 25.75
S | Sell | 30.8 | 
Code: P Type Error Profit: 2.4499999999999993
P | Sell | 30.8 | 
Code: C Type Error Code: 1102
C | Sell | 30.8 | 
Code: S Type Error Sell: 29.1
S | Sell | 30.8 | 
Code: P Type Error Profit: -0.9000000000000021
P | Sell | 30.8 | 
Code: C Type Error Code: 1102
C | Sell | 30.8 | 
Code: S Type Error Sell: 29.0
S | Sell | 30.8 | 
Code: P Type Error Profit: -0.8000000000000007
P | Sell | 30.8 | 
Code: C Type Error C

In [3]:
def getListedCode(typ="twse",codeLength=4):
    import sqlite3
    sqlite_file = "list.db" #上市上櫃下市 股票代號列表 資料庫檔案
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    out = conn.execute("SELECT code, type FROM StockList WHERE type LIKE ? AND length(code) == ? ",("twse",codeLength)) 
    # 取 代號4碼 類別上市 股票代號
    codeList = []
    for i, row in enumerate(out):
        codeList.append(row[0])
    conn.close()
    print("取得股票代號總數:", len(codeList)) #codeList 為取出股票代號列表
    return codeList

def getProfits(code):
    import sqlite3
    conn = sqlite3.connect("ProfitCosty1617.db")
    c = conn.cursor()
    out = conn.execute("SELECT * FROM twseprofit WHERE code LIKE ?",(code,))
    dataList = []
    for i in out:
        dataList.append(i)
    conn.close()
    return dataList

codeList = getListedCode()

roiByCode = []
errCost = 0
posProfit = 0
negProfit = 0
for code in codeList:
    dataList = getProfits(code)
    roi = []
    totalProfit = 0
    transactionx = 0
    for i in dataList:
        cost = i[3]
        profit = i[4]
        if not cost:
            #print(i, cost)
            errCost = errCost + 1
        else:
            if profit > 0: posProfit = posProfit + 1
            if profit < 0: negProfit = negProfit + 1
            roi = profit/cost
            roiByCode.append(["Code:{}".format(code),"ROI:{}".format(roi)])

print(roiByCode[0])

取得股票代號總數: 913
['Code:1101', 'ROI:-0.048814504881450484']


In [4]:

print(roiByCode[0:5])

NameError: name 'sumProfit' is not defined

In [6]:
import pickle
pickle.dump(roiByCode,open('roiByCode.pkl', 'wb'))

In [26]:
quoteRoi = []
for i in roiByCode:
    quoteRoi.append([i[0],i[1]])
#print(max(quoteRoi, key=lambda item: item[1]))

print(roiByCode[0])

['Code:1101', 'Sum:-0.048814504881450484', 'Mean:-0.048814504881450484', 'Max:-0.048814504881450484', 'Min:-0.048814504881450484']


In [1]:
import pickle
roiByCode = pickle.load(open('roiByCode.pkl', 'rb'))

def createDatabase(dbName,tableName):
    import os
    if os.path.isfile(dbName):
        return "Database name already exists: ", dbName
    else:    
        import sqlite3
        sqlite_file = dbName
        conn = sqlite3.connect(sqlite_file)
        c = conn.cursor()
        c.execute('CREATE TABLE {tn} (id INTEGER PRIMARY KEY, code TEXT NOT NULL, roi REAL)'.format(tn=tableName)) 
        conn.commit()
        conn.close()
        return "Database",dbName," Successfully Created!"
    
def insertData(dbName, tableName, roiByCode):
    import sqlite3
    conn = sqlite3.connect(dbName)
    c = conn.cursor()
    count = 0
    for code, roi in roiByCode:
        conn.execute("INSERT INTO twseROI (code, roi) VALUES (?,?)",(code[5:], roi[4:]))
        count = count + 1
    conn.commit()
    conn.close()
    return count
    
    
createDatabase("ROIbyCode.db","twseROI")
insertData("ROIbyCode.db", "twseROI", roiByCode)


7800

In [16]:
def getStrategyData(code,startdate,enddate,dbname="buysell.db",table="twse"):
    import sqlite3
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    out = conn.execute("SELECT * FROM twse WHERE code LIKE ? AND date between ? and ?",(code,startdate,enddate))
    dataList = []
    for i in out:
        dataList.append(i)
    conn.close()
    return dataList

dataList = getStrategyData(code,"2016-05-25","2017-05-25")
print(dataList[0][1])

9188


In [2]:
def getLastDate(code,dbname="PriceVolData.db",table="twse"):
    import sqlite3
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    out = conn.execute("SELECT * FROM twse WHERE code LIKE ? ORDER BY date DESC LIMIT 1",(code,))
    dataList = []
    for i in out:
        dataList.append(i)
    conn.close()
    return dataList

l = getLastDate(1101)
print(l)

[(2455, '1101', '2017-05-26 13:30:00', 34.65, 34.85, 34.55, 34.7, 4339000.0)]


In [6]:
import pickle
pickle.dump(profitList, open( "profitList.pkl", "wb" ) )
profList = pickle.load( open( "profitList.pkl", "rb" ) )

for i in profList[0:5]:
    print(i)
    
print("勝率:", winSum/(loseSum+winSum))
    
#取得股票代號總數: 913
#Profit Sum: 12492.109999999988 Win Sum: 3373 Lose Sum: 2916

['Code:1101', 'Profit:-9.599999999999973']
['Code:1102', 'Profit:-1.3500000000000014']
['Code:1103', 'Profit:0.9000000000000004']
['Code:1104', 'Profit:14.299999999999994']
['Code:1108', 'Profit:1.9000000000000004']
勝率: 0.5605313092979127


In [8]:
def createTable(dbName,tableName):
    import os
    import sqlite3
    sqlite_file = dbName
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    c.execute('CREATE TABLE {tn} (indexColumn INTEGER PRIMARY KEY, code TEXT NOT NULL, profit REAL NOT NULL)'.format(tn=tableName)) 
    conn.commit()
    conn.close()
    return "Table",tableName," Successfully Created!"
print("createTable loaded")

dbName = "PriceVolData.db"
tableName = "profit"
print(createTable(dbName,tableName))

createTable loaded
('Table', 'profit', ' Successfully Created!')


In [29]:
def insertData(dbName, tableName, dataList):
    import sqlite3
    conn = sqlite3.connect(dbName)
    c = conn.cursor()
    count = 0
    for code, profit in dataList:
        conn.execute("INSERT INTO profit (code, profit) VALUES (?,?)",(code, profit))
        count = count + 1
    conn.commit()
    conn.close()
    return count
print("insertData loaded")

dbName = "PriceVolData.db"
tableName = "profit"
import pickle
pickle.dump(profitList, open( "profitList.pkl", "wb" ) )
profList = pickle.load( open( "profitList.pkl", "rb" ) )

newDataList = []
for i in profList:
    newDataList.append([str(i[0])[7:-2],str(i[1])[9:-2]])
insertData(dbName, tableName, newDataList)

insertData loaded


913